# data frame API
based on age predit salary

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ML1').getOrCreate()

In [2]:
training = spark.read.csv('Test5.csv',header=True,inferSchema=True)

In [3]:
training.show()

+---+-------+---+-------+-----+---+
| id|   name|age|    sal| dept|exp|
+---+-------+---+-------+-----+---+
|  1| Anvesh| 22|  32113|  CSE|  1|
|  2|  Surya| 22| 223212|CIVIL|  2|
|  3| nikhil| 25| 234222|  CSE|  4|
|  4|dheeraj| 22|  12311|CIVIL|  1|
|  5|karthik| 23|6516516| MECH|  5|
+---+-------+---+-------+-----+---+



In [4]:
training.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- sal: integer (nullable = true)
 |-- dept: string (nullable = true)
 |-- exp: integer (nullable = true)



In [5]:
training.columns

['id', 'name', 'age', 'sal', 'dept', 'exp']

In [6]:
# [AGE,SAL] new independent feature

In [7]:
from pyspark.ml.feature import  VectorAssembler
featureassebler = VectorAssembler(inputCols=['age','exp'],outputCol='independentcol')

In [8]:
output = featureassebler.transform(training)

In [9]:
output.show()

+---+-------+---+-------+-----+---+--------------+
| id|   name|age|    sal| dept|exp|independentcol|
+---+-------+---+-------+-----+---+--------------+
|  1| Anvesh| 22|  32113|  CSE|  1|    [22.0,1.0]|
|  2|  Surya| 22| 223212|CIVIL|  2|    [22.0,2.0]|
|  3| nikhil| 25| 234222|  CSE|  4|    [25.0,4.0]|
|  4|dheeraj| 22|  12311|CIVIL|  1|    [22.0,1.0]|
|  5|karthik| 23|6516516| MECH|  5|    [23.0,5.0]|
+---+-------+---+-------+-----+---+--------------+



In [10]:
output.columns

['id', 'name', 'age', 'sal', 'dept', 'exp', 'independentcol']

In [11]:
finalizedData = output.select('independentcol','sal')

In [12]:
finalizedData.show()

+--------------+-------+
|independentcol|    sal|
+--------------+-------+
|    [22.0,1.0]|  32113|
|    [22.0,2.0]| 223212|
|    [25.0,4.0]| 234222|
|    [22.0,1.0]|  12311|
|    [23.0,5.0]|6516516|
+--------------+-------+



In [ ]:
from pyspark.ml.regression import  LinearRegression


trainData , testData = finalizedData.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='independentcol',labelCol='sal')
regressor = regressor.fit(trainData)

In [15]:
regressor.coefficients

DenseVector([-1793368.2857, 2090816.2338])

In [16]:
regressor.intercept

36856349.50648801

In [20]:
predRess = regressor.evaluate(testData).show()

IllegalArgumentException: requirement failed: Nothing has been added to this summarizer.

In [21]:
from pyspark.ml.regression import LinearRegression

# Split the data
trainData, testData = finalizedData.randomSplit([0.75, 0.25])

# Train the model
regressor = LinearRegression(featuresCol='independentcol', labelCol='sal')
model = regressor.fit(trainData)

# Evaluate the model
summary = model.evaluate(testData)

# Access metrics
print("Root Mean Squared Error (RMSE):", summary.rootMeanSquaredError)
print("R^2:", summary.r2)
print("Mean Absolute Error (MAE):", summary.meanAbsoluteError)

# If you want to see the predictions
summary.predictions.show()

# Access the coefficients
print("Coefficients:", model.coefficients)


Root Mean Squared Error (RMSE): 4468773.941131168
R^2: -1.0168766640364053
Mean Absolute Error (MAE): 3241646.9999999995
+--------------+-------+-----------------+
|independentcol|    sal|       prediction|
+--------------+-------+-----------------+
|    [22.0,2.0]| 223212|57546.99999999988|
|    [23.0,5.0]|6516516|198887.0000000007|
+--------------+-------+-----------------+

Coefficients: [35335.00000000029,35335.00000000015]
